In [5]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os

from astroquery.gaia import Gaia 

# DATASET

Dataset de ***RR Lyrae*** y de ***Eclipse binaria***:
- **N** ejemplos cada uno
- periodo de catálogo no nulo
- al menos **L** puntos en cada banda
- magnitud promedio en banda g menor a **M**.

In [6]:
def dataset(N, L, M):

    # Crear carpeta dataset
    directory = "dataset"
    parent_dir = "../../PeriodogramsGaia/"
    path = os.path.join(parent_dir, directory)  
    os.mkdir(path) 

    # Lista de directorios
    directories = ['ECL', 'RR']
    # crea carpetas para cada directories
    for directory in directories:
        parent_dir = "../../PeriodogramsGaia/dataset"
        path = os.path.join(parent_dir, directory)  
        os.mkdir(path)

    # Crea dataFrame con source_id, periodo y tipo
    data_global = pd.DataFrame()



    for tabla in ["vari_eclipsing_binary","vari_rrlyrae"]:
        ids_stars = []
        NN = 0
        while (NN < N):
        
            if tabla == "vari_rrlyrae":
                # Hace la consulta
                query = f"""
                select top { N-NN } *
                from gaiadr3.{tabla}
                where pf IS NOT NULL
                """
                type_star = "RR"
            else:
                query = f"""
                select top { N-NN } *
                from gaiadr3.{tabla}
                where frequency IS NOT NULL
                order by global_ranking desc
                """
                type_star = "ECL"
            job = Gaia.launch_job(query)
            ids = job.get_results().to_pandas("source_id")
            # Extrae las curvas de Gaia DR3
            datalink = Gaia.load_data(ids=ids.index, 
                            data_release='Gaia DR3', 
                            retrieval_type='EPOCH_PHOTOMETRY', 
                            format='csv', 
                            valid_data=False)
            type(datalink), len(datalink)
            # Evalua las curvas
            for key, value in datalink.items():
                '''
                points_BP = lc.groupby(['band']).count().source_id['BP']
                points_G = lc.groupby(['band']).count().source_id['G']
                points_RP = lc.groupby(['band']).count().source_id['RP']
                mag_mean_G = lc.groupby(['band']).mean().mag['G'] '''

                lc = value[0].to_pandas()
                points_G = lc.groupby("band").get_group("G").shape[0]
                points_BP = lc.groupby("band").get_group("BP").shape[0]
                points_RP = lc.groupby("band").get_group("RP").shape[0]
                mag_mean_G = np.mean(lc.groupby("band").get_group("G")["mag"])

                name = lc.source_id[0]
                # Ver si cumple las condiciones de puntos en cada banda, mag promedio en la banda G y que no este repetido
                if ((points_BP >= L) & (points_G >= L) & (points_RP >= L) & (mag_mean_G < M) & (name not in ids_stars)):
                    ids_stars.append(name)
                    NN += 1
                    # Creo carpeta para cada lc
                    direct = "../../PeriodogramsGaia/dataset"
                    parent_dir = os.path.join(direct, type_star)
                    path = os.path.join(parent_dir, str(name))
                    os.mkdir(path) 
                    path_lc = os.path.join(path, str(name)+'.pkl')
                    lc.to_pickle(path_lc)
        
        # DataFrame con pf, tipo y source_id
        if type_star == "ECL":
            query = f"""
                select source_id,frequency
                from gaiadr3.{tabla}
                where source_id IN {tuple(ids_stars)}
            """

            job = Gaia.launch_job(query)
            info_star_type = job.get_results().to_pandas("source_id")
            info_star_type["frequency"] = 1/info_star_type["frequency"]
            info_star_type = info_star_type.rename(columns= {"frequency":"pf"})
        else:
            query = f"""
                select source_id,pf
                from gaiadr3.{tabla}
                where source_id IN {tuple(ids_stars)}
            """
            job = Gaia.launch_job(query)
            info_star_type = job.get_results().to_pandas("source_id")
        info_star_type["type"] = type_star
        data_global = pd.concat([data_global,info_star_type])

    return data_global

In [8]:
data_global = dataset(100,5,100)

In [9]:
data_global.to_pickle("../../PeriodogramsGaia/dataset/data_global.pkl")

In [10]:
data_global

,pf,type
source_id,,
184457373509254272,3.406982,ECL
187514458214460928,0.689039,ECL
193720243577690880,3.335009,ECL
204067060310941696,1.849591,ECL
251636125219970816,0.426586,ECL
...,...,...
5902591804246642944,0.653311,RR
5902653514326537472,0.572423,RR
5937091180561923968,0.574506,RR
